In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import mapclassify as mp
from shapely.geometry import Point, Polygon
import datetime as dt

### FUENTES:
#### MAPAS:
https://www.ign.gob.ar/NuestrasActividades/InformacionGeoespacial/CapasSIG
#### BASE COVID:
https://datos.gob.ar/dataset/salud-covid-19-casos-registrados-republica-argentina/archivo/salud_fd657d02-a33a-498b-a91b-2ef1a68b8d16

In [ ]:
dptos=gpd.read_file("C:/Users/dani/Desktop/PROYECTOS/MAPAS/departamento/departamento.shp")
provs=gpd.read_file("C:/Users/dani/Desktop/PROYECTOS/MAPAS/provincia/provincia.shp")
covid=pd.read_csv("C:/Users/dani/Desktop/PROYECTOS/DATOS ABIERTOS ARGENTINA/Covid19Casos.csv")

Se elige las jurisdicciones a plotear en el mapa y a estudiar en la base, en este caso Salta y Jujuy

In [ ]:
a_mapa=['Salta','Jujuy']
mapa=provs.loc[provs['nam'].isin(a_mapa)]
noa=covid.loc[covid['residencia_provincia_nombre'].isin(a_mapa)]

In [ ]:
mapa.plot()

intersección entre el mapa provincial y de departamentos

In [ ]:
mapa=gpd.overlay(mapa,dptos,how='intersection')

eliminamos las filas con los departamentos fantasmas del ign

In [ ]:
no=['Copo', 'Matacos']
mapa=mapa[~mapa.nam_2.isin(no)]
#duplicado de gral güemes
mapa = mapa[mapa.gid_2 != 20]

In [ ]:
mapa.plot()

eliminamos las columnas que no nos sirven de mapas

In [ ]:
mapa.drop(['gid_1', 'entidad', 'objeto_1', 'gna_1','fna_1','in1_1', 'fdc_1', 'sag_1',
           'gid_2', 'objeto_2', 'gna_2','fna_2', 'fdc_2', 'sag_2'],axis=1, inplace=True)

In [ ]:
mapa.head()

Usamos el módulo datetime para cambiar la string con la fecha de casos a un dato de tiempo y creamos la columna semana que nos dirá la semana del año correspondiente a cada dato

In [ ]:
pd.options.mode.chained_assignment = None
noa['fecha_diagnostico'] =  pd.to_datetime(noa['fecha_diagnostico'], format='%Y-%m-%d')

In [ ]:
noa['semana']=noa['fecha_diagnostico'].dt.isocalendar().week 
noa['semana'] = noa['semana'].astype('Int64')

para poder hacer un join de datasets usando como key el nombre del departamento hay que hacer algunos cambios, en este caso en el nombre del departamento San Martín

In [ ]:
mapa['nam_2'].replace('General José de San Martín', 'San Martín', inplace=True)

En esta base están casos confirmados y descartados asi que deben seleccionarse las categorías deseadas

In [ ]:
noa['clasificacion'].unique()

In [ ]:
conf_lista=['Caso confirmado por laboratorio - No Activo por criterio de laboratorio','Caso confirmado - No activo (por tiempo de evolución)',
    'Caso confirmado por laboratorio - Activo Internado','Caso confirmado - Fallecido', 'Caso confirmado - Activo']

confirmados=noa.loc[noa['clasificacion'].isin(conf_lista)]


In [ ]:
noa['residencia_departamento_nombre']=noa['residencia_departamento_nombre'].replace('Grl. José de San Martín', 'San Martín')
noa['residencia_departamento_nombre']=noa['residencia_departamento_nombre'].replace('SIN ESPECIFICAR', np.NaN)
noa['residencia_departamento_nombre']=noa['residencia_departamento_nombre'].dropna()

realizamos un group by y una tabla pivot para obtener los casos por semana y dpto

In [ ]:
dptos_semana=confirmados.groupby(['residencia_departamento_nombre', 'semana']).size().reset_index(name='casos').pivot(columns='semana', index='residencia_departamento_nombre', values='casos').fillna(0)

a este nuevo dataframe le debemos agregar como columnas a las semanas que no han tenido ningún caso primero y una vez hecho esto con todas las columnas restantes asignarles el numero de fecha a la que corresponde cada semana

In [ ]:
#se crea lista con nombre de columnas
listin=dptos_semana.columns.tolist()

#crear lista con el range min max de la primera, los numeros debe
listin2=[*range(int(min(listin)),int(max(listin)+1))]

#identificar los valores unicos entre listas, estos serán las colu,mnas que deseamos agregar rellenas de 0
nuevas_cols=list((set(listin) | set(listin2)) - (set(listin) & set(listin2)))

#creamos las columnas en los dataframes, van rellenas con ceros
for col in nuevas_cols:
    dptos_semana[col] = 0
#ordenamos las columnas por nombre
dptos_semana = dptos_semana.reindex(sorted(dptos_semana.columns), axis=1)


#renombramos las columnas usando el modulo datetime que nos de la fecha de los lunes de cada semana

nsem=(max(dptos_semana.columns))
today = dt.date.today()
for i in range(max(dptos_semana.columns)+1):
    l=i-nsem
    name= str(today + dt.timedelta(days=-today.weekday(), weeks=l))
    dptos_semana = dptos_semana.rename(columns={i:name})


In [ ]:
dptos_semana.columns

Ahora podemos unir la tabla pivot generada con el dataframe de geopandas y camnbiamos los nan por 0 para poder graficar correctamente los departamentos sin casos

In [ ]:
semanal = mapa.merge(dptos_semana, left_on='nam_2', right_on='residencia_departamento_nombre', how='left')

In [ ]:
semanal=semanal.fillna(0)

En este caso en particular decidí fijar los bins que yo deseaba para una mejor comparación ya que mapclassify los va cambiando según la estructura de datos, pero puede hacerse de otra forma. Para otras opciones:
https://geopandas.org/gallery/plotting_with_geoplot.html

In [ ]:
co=semanal.iloc[:,4:].columns
cut_labels_6 = ['sin casos','1 a9', '10 a 49', '50 a 99', '100 a 199', '200 a 399', '400 a 800']
cut_bins = [0,1, 10, 50, 100, 200,400,800]

semanal[co] =semanal[co].apply(pd.cut, bins=cut_bins, labels=cut_labels_6, right=False)
#semanal.iloc[:,4:]=semanal.iloc[:,4:].astype(int)

creamos el plot y hacemos un loop por las columnas con los datos semanales

In [ ]:
path='C:/Users/dani/Desktop/PROYECTOS/source_images/'
fig = plt.subplots()
i=0
for col in semanal.columns[4:]:    
    fig, ax = plt.subplots(1, figsize=(15,10))
    semanal.plot(categorical=True, ax=ax,column=col,cmap='OrRd',legend=True,  linewidth=1,legend_kwds={'loc':3},
             edgecolor='black')    
    title_obj = plt.title(str(col) + '\nCasos semanales: '+ str(round(sum(dptos_semana.iloc[:,i])))) 
    plt.setp(title_obj, color='black', fontsize=20 )
    plt.savefig(fname=path+col+".png",dpi=100,bbox_inches='tight' )
    #closefig sirve para no mostrar la imagen en el notebook ya que en loops grandes aparece un warning
    #de que hay muchas pestañas abiertas, puede comentarse esta linea para ver los mapas
    plt.close(fig)
    i+=1


importamos imageio para generar el gif

In [ ]:
import imageio

accedemos a las imagenes para el gif

In [ ]:
from pathlib import Path
image_path = Path("C:/Users/dani/Desktop/PROYECTOS/source_images")
images = list(image_path.glob('*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))

lo generamos pasando un path a donde deseemos que se guarde

In [ ]:
path="C:/Users/dani/Desktop/PROYECTOS/source_images"

In [ ]:
imageio.mimwrite(path+'Salta_covid_semanal.gif', image_list, fps=1.5)